In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\apex\\project\\Predicting_Bitcoin_Prices_Using_ANN\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\apex\\project\\Predicting_Bitcoin_Prices_Using_ANN'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from ANNClassifier.constants import *
# from ANNClassifier.utils.common import read_yaml

from src.ANNClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_URL= config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir=config.unzip_dir
            )

            return data_ingestion_config


In [8]:
import os
import zipfile
import gdown
from ANNClassifier import logger
from ANNClassifier.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [26]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-23 13:34:56,300: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-23 13:34:56,303: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-23 13:34:56,305: INFO: common: created directory at: artifacts]
[2025-07-23 13:34:56,307: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-23 13:34:56,308: INFO: 1235392990: Downloading data from https://drive.google.com/file/d/1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T
From (redirected): https://drive.google.com/uc?export=download&id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T&confirm=t&uuid=5b8f77a3-8a6e-485b-b608-ab7c45576ebb
To: c:\Users\HP\Desktop\apex\project\Predicting_Bitcoin_Prices_Using_ANN\artifacts\data_ingestion\data.zip
100%|██████████| 369M/369M [00:36<00:00, 10.2MB/s] 


[2025-07-23 13:35:39,783: INFO: 1235392990: Downloaded data from https://drive.google.com/file/d/1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T/view?usp=sharing into file artifacts/data_ingestion/data.zip]


BadZipFile: File is not a zip file

In [9]:
import os
import gdown
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from logging import getLogger
from typing import Optional

In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> None:
        try:
            dataset_url = self.config.source_URL
            download_path = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)

            logger.info(f"Downloading CSV from {dataset_url} into {download_path}")
            gdown.download(dataset_url, str(download_path), quiet=False, fuzzy=True)
            logger.info(f"Download completed: {download_path}")
        except Exception as e:
            logger.error("Download failed")
            raise e

    def load_csv(self) -> pd.DataFrame:
        try:
            logger.info(f"Loading CSV file from {self.config.local_data_file}")
            df = pd.read_csv(self.config.local_data_file)
            logger.info(f"CSV file loaded with shape: {df.shape}")
            return df
        except Exception as e:
            logger.error("Failed to load CSV")
            raise e

In [13]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-30 19:12:02,923: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-30 19:12:02,947: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-30 19:12:02,950: INFO: common: created directory at: artifacts]
[2025-07-30 19:12:02,953: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-30 19:12:02,955: INFO: 2364431818: Downloading CSV from https://drive.google.com/file/d/1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T/view?usp=sharing into artifacts/data_ingestion/data.csv]


Downloading...
From (original): https://drive.google.com/uc?id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T
From (redirected): https://drive.google.com/uc?id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T&confirm=t&uuid=93c0fabd-0197-49f9-8fa6-425db9194a1c
To: c:\Users\HP\Desktop\apex\project\Predicting_Bitcoin_Prices_Using_ANN\artifacts\data_ingestion\data.csv
100%|██████████| 369M/369M [02:19<00:00, 2.65MB/s] 


[2025-07-30 19:14:30,283: INFO: 2364431818: Download completed: artifacts/data_ingestion/data.csv]


AttributeError: 'DataIngestion' object has no attribute 'extract_zip_file'

In [ ]:
try:
    # Step 1: Load config
    config = ConfigurationManager()

    # Step 2: Get data ingestion configuration
    data_ingestion_config = config.get_data_ingestion_config()

    # Step 3: Initialize ingestion
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Step 4: Download the CSV file
    data_ingestion.download_file()

    # Step 5: Load the CSV into a DataFrame
    df = data_ingestion.load_csv()

    # Optional: Preview or log DataFrame shape
    print(df.head())  # or logger.info(df.head())
    print(f"✅ CSV loaded successfully. Shape: {df.shape}")

except Exception as e:
    print(f"❌ Error during data ingestion: {e}")
    raise e


[2025-07-23 18:20:14,278: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-23 18:20:14,280: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-23 18:20:14,282: INFO: common: created directory at: artifacts]
[2025-07-23 18:20:14,283: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-23 18:20:14,285: INFO: 2364431818: Downloading CSV from https://drive.google.com/file/d/1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T/view?usp=sharing into artifacts/data_ingestion/data.csv]


Downloading...
From (original): https://drive.google.com/uc?id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T
From (redirected): https://drive.google.com/uc?id=1zbilaliA_PAH7xMBGsrgYEuwb-PzvS6T&confirm=t&uuid=6646bd87-dbc7-42be-890d-9bcb00af01f9
To: c:\Users\HP\Desktop\apex\project\Predicting_Bitcoin_Prices_Using_ANN\artifacts\data_ingestion\data.csv
100%|██████████| 369M/369M [00:32<00:00, 11.4MB/s] 

[2025-07-23 18:20:49,264: INFO: 2364431818: Download completed: artifacts/data_ingestion/data.csv]
[2025-07-23 18:20:49,265: INFO: 2364431818: Loading CSV file from artifacts/data_ingestion/data.csv]


[2025-07-23 18:20:52,269: INFO: 2364431818: CSV file loaded with shape: (7073005, 6)]
      Timestamp  Open  High   Low  Close  Volume
0  1.325412e+09  4.58  4.58  4.58   4.58     0.0
1  1.325412e+09  4.58  4.58  4.58   4.58     0.0
2  1.325412e+09  4.58  4.58  4.58   4.58     0.0
3  1.325412e+09  4.58  4.58  4.58   4.58     0.0
4  1.325412e+09  4.58  4.58  4.58   4.58     0.0
✅ CSV loaded successfully. Shape: (7073005, 6)
